## Install TensorTrade

In [7]:
!python3 -m pip install -e .. -U

Obtaining file:///Users/ronaldopontes/Google%20Drive/code/tensortrade
  Attempting uninstall: tensortrade
    Found existing installation: tensortrade 0.2.0b1
    Uninstalling tensortrade-0.2.0b1:
      Successfully uninstalled tensortrade-0.2.0b1
  Running setup.py develop for tensortrade


## Install dependencies for this example notebook


In [8]:
!python3 -m pip install ta

Processing /Users/ronaldopontes/Library/Caches/pip/wheels/50/b3/df/5daa3fb600f28142ea289eb27a3982da44bfc70c1362d470ee/ta-0.5.18-py3-none-any.whl


## Setup Data Fetching

In [9]:
import ssl
import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context # Only used if pandas gives a SSLError

def fetch(exchange_name, symbol, timeframe):
    url = "https://www.cryptodatadownload.com/cdd/"
    filename = "{}_{}USD_{}.csv".format(exchange_name, symbol, timeframe)
    volume_column = "Volume {}".format(symbol)
    new_volume_column = "Volume_{}".format(symbol)
    
    df = pd.read_csv(url + filename, skiprows=1)
    df = df[::-1]
    df = df.drop(["Symbol"], axis=1)
    df = df.rename({"Volume USD": "volume", volume_column: new_volume_column}, axis=1)
    df = df.set_index("Date")
    df.columns = [symbol + ":" + name.lower() for name in df.columns]
                     
    return df

## Fetch Historical Data

In [10]:
coinbase_data = pd.concat([
    fetch("Coinbase", "BTC", "1h"),
    fetch("Coinbase", "ETH", "1h")
], axis=1)

bitstamp_data = pd.concat([
    fetch("Bitstamp", "BTC", "1h"),
    fetch("Bitstamp", "LTC", "1h")
], axis=1)

In [11]:
coinbase_data.head()

,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume_btc,BTC:volume,ETH:open,ETH:high,ETH:low,ETH:close,ETH:volume_eth,ETH:volume
Date,,,,,,,,,,,,
2017-07-01 11-AM,2505.56,2513.38,2495.12,2509.17,114.60,287000.32,279.98,279.99,272.1,275.01,2455.28,679358.87
2017-07-01 12-PM,2509.17,2512.87,2484.99,2488.43,157.36,393142.50,275.01,275.01,271.0,274.83,3023.14,824362.87
2017-07-01 01-PM,2488.43,2488.43,2454.40,2454.43,280.28,693254.01,274.83,274.93,265.0,268.79,11204.43,3010787.99
2017-07-01 02-PM,2454.43,2473.93,2450.83,2459.35,289.42,712864.80,268.79,269.90,265.0,265.74,6367.05,1702536.85
2017-07-01 03-PM,2459.35,2475.00,2450.00,2467.83,276.82,682105.41,265.74,272.74,265.0,272.57,5581.66,1500282.55


In [12]:
bitstamp_data.head()

,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume_btc,BTC:volume,LTC:open,LTC:high,LTC:low,LTC:close,LTC:volume_ltc,LTC:volume
Date,,,,,,,,,,,,
2017-07-01 11-AM,2506.50,2510.62,2495.50,2500.00,208.52,521903.70,39.67,39.67,39.32,39.45,49.61,1957.48
2017-07-01 12-PM,2500.00,2503.69,2488.25,2495.62,243.08,607308.42,39.45,39.57,39.18,39.57,63.62,2507.48
2017-07-01 01-PM,2495.62,2495.62,2433.59,2449.01,579.13,1421546.81,39.57,39.57,38.80,38.90,564.39,22019.12
2017-07-01 02-PM,2449.01,2480.60,2438.88,2457.90,528.75,1302198.58,38.90,39.08,38.77,39.06,788.53,30732.98
2017-07-01 03-PM,2457.90,2471.88,2452.10,2466.35,292.31,719666.88,39.06,39.16,38.73,39.15,666.53,26053.18


## Define Exchanges

An exchange needs a name, an execution service, and streams of price data in order to function properly.

The setups supported right now are the simulated execution service using simulated or stochastic data. More execution services will be made available in the future, as well as price streams so that live data and execution can be supported.

In [13]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream

coinbase = Exchange("coinbase", service=execute_order)(
    Stream("USD-BTC", list(coinbase_data['BTC:close'])),
    Stream("USD-ETH", list(coinbase_data['ETH:close']))
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream("USD-BTC", list(bitstamp_data['BTC:close'])),
    Stream("USD-LTC", list(bitstamp_data['LTC:close']))
)

Now that the exchanges have been defined we can define our features that we would like to include, excluding the prices we have provided for the exchanges.

## Define External Data Feed

Here we will define the external feed to use whatever data you would like. From financial indicators to datasets that have nothing to do with instrument prices, they will all have to be defined and incorporated into the external data feed provided to the environment. There is also an internal data feed that is used to collect data on all the wallets and the net worth of the portfolio. You can choose to include this or not by using the `use_internal` parameter of the trading environment.

In [14]:
import ta

from tensortrade.data import DataFeed, Module

# Add all features for coinbase bitcoin
coinbase_btc = coinbase_data.loc[:, [name.startswith("BTC") for name in coinbase_data.columns]]
coinbase_eth = coinbase_data.loc[:, [name.startswith("ETH") for name in coinbase_data.columns]]

ta.add_all_ta_features(
    coinbase_btc,
    colprefix="BTC:",
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)


with Module("coinbase") as coinbase_ns:
    coinbase_nodes = [Stream(name, list(coinbase_btc[name])) for name in coinbase_btc.columns]
    coinbase_nodes += [Stream(name, list(coinbase_eth[name])) for name in coinbase_eth.columns]
    

# Add all features for coinbase ethereum
bitstamp_btc = bitstamp_data.loc[:, [name.startswith("BTC") for name in bitstamp_data.columns]]  
bitstamp_ltc = bitstamp_data.loc[:, [name.startswith("LTC") for name in bitstamp_data.columns]]

ta.add_all_ta_features(
    bitstamp_ltc,
    colprefix="LTC:",
    **{k: "LTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with Module("bitstamp") as bitstamp_ns:
    bitstamp_nodes = [Stream(name, list(bitstamp_btc[name])) for name in bitstamp_btc.columns]
    bitstamp_nodes += [Stream(name, list(bitstamp_ltc[name])) for name in bitstamp_ltc.columns]


feed = DataFeed([coinbase_ns, bitstamp_ns])

In [15]:
feed.next()

{'coinbase:/BTC:open': 2505.56,
 'coinbase:/BTC:high': 2513.38,
 'coinbase:/BTC:low': 2495.12,
 'coinbase:/BTC:close': 2509.17,
 'coinbase:/BTC:volume_btc': 114.6,
 'coinbase:/BTC:volume': 287000.32,
 'coinbase:/BTC:volume_adi': 154659.5371741516,
 'coinbase:/BTC:volume_obv': 287000.32,
 'coinbase:/BTC:volume_cmf': 0.5388828039430464,
 'coinbase:/BTC:volume_fi': nan,
 'coinbase:/BTC:momentum_mfi': nan,
 'coinbase:/BTC:volume_em': nan,
 'coinbase:/BTC:volume_sma_em': nan,
 'coinbase:/BTC:volume_vpt': -189074.94639155112,
 'coinbase:/BTC:volume_nvi': 1000.0,
 'coinbase:/BTC:volume_vwap': 2505.89,
 'coinbase:/BTC:volatility_atr': 0.0,
 'coinbase:/BTC:volatility_bbm': 2509.17,
 'coinbase:/BTC:volatility_bbh': 2509.17,
 'coinbase:/BTC:volatility_bbl': 2509.17,
 'coinbase:/BTC:volatility_bbw': 0.0,
 'coinbase:/BTC:volatility_bbp': nan,
 'coinbase:/BTC:volatility_bbhi': 0.0,
 'coinbase:/BTC:volatility_bbli': 0.0,
 'coinbase:/BTC:volatility_kcc': 2505.89,
 'coinbase:/BTC:volatility_kch': 2524.

## Portfolio

Make the portfolio using the any combinations of exchanges and intruments that the exchange supports

In [16]:
from tensortrade.instruments import USD, BTC, ETH, LTC
from tensortrade.wallets import Wallet, Portfolio

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 10 * BTC),
    Wallet(coinbase, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 3 * LTC),
])

## Environment

In [17]:
from tensortrade.environments import TradingEnvironment

env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    action_scheme='managed-risk',
    reward_scheme='simple',
    window_size=15,
    enable_logger=False
)

In [18]:
env.feed.next()

{'coinbase:/BTC:open': 2505.56,
 'coinbase:/BTC:high': 2513.38,
 'coinbase:/BTC:low': 2495.12,
 'coinbase:/BTC:close': 2509.17,
 'coinbase:/BTC:volume_btc': 114.6,
 'coinbase:/BTC:volume': 287000.32,
 'coinbase:/BTC:volume_adi': 154659.5371741516,
 'coinbase:/BTC:volume_obv': 287000.32,
 'coinbase:/BTC:volume_cmf': 0.5388828039430464,
 'coinbase:/BTC:volume_fi': nan,
 'coinbase:/BTC:momentum_mfi': nan,
 'coinbase:/BTC:volume_em': nan,
 'coinbase:/BTC:volume_sma_em': nan,
 'coinbase:/BTC:volume_vpt': -189074.94639155112,
 'coinbase:/BTC:volume_nvi': 1000.0,
 'coinbase:/BTC:volume_vwap': 2505.89,
 'coinbase:/BTC:volatility_atr': 0.0,
 'coinbase:/BTC:volatility_bbm': 2509.17,
 'coinbase:/BTC:volatility_bbh': 2509.17,
 'coinbase:/BTC:volatility_bbl': 2509.17,
 'coinbase:/BTC:volatility_bbw': 0.0,
 'coinbase:/BTC:volatility_bbp': nan,
 'coinbase:/BTC:volatility_bbhi': 0.0,
 'coinbase:/BTC:volatility_bbli': 0.0,
 'coinbase:/BTC:volatility_kcc': 2505.89,
 'coinbase:/BTC:volatility_kch': 2524.